In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from loader import Loader
from constant import FeatureType, FileType
from copy import deepcopy
from skimage.measure import compare_nrmse
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, LSTM, Dense, RepeatVector
import time
import argparse
import numpy as np
import tensorflow as tf
import hashlib
import json

### Train, Validate, Test

In [3]:
def encoder_decoder(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']

    batch_size = args.batch_size
    epochs = args.epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stop)

    with tf.device('/GPU:0'):        
        encoder_inputs = Input(shape=(len(args.features), args.x_frames * 24))

        encoder = LSTM(args.hid_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]

        repeat_vector = RepeatVector(24)
        repeat_vector_outputs = repeat_vector(encoder_outputs)

        decoder_lstm = LSTM(args.hid_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(repeat_vector_outputs) # , initial_state=encoder_states)

        decoder_dense = Dense(1, activation=args.activation)
        decoder_outputs = decoder_dense(decoder_outputs)

        rmsprop = RMSprop(lr=args.lr)

        model = Model(encoder_inputs, decoder_outputs)
        model.compile(optimizer=rmsprop, loss=tf.keras.losses.MeanSquaredError(), metrics='accuracy')

        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=batch_size, epochs=epochs, callbacks=[callback])
        
        scores = model.evaluate(X_val, y_val, verbose=0)
        
        print('LSTM validation scores:', scores)
        print(model.summary())

    return model, history

In [4]:
def test(partition, model, args):
    X_test, y_test = partition['test']
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape(y_test.shape[0], y_test.shape[1])

    nrmse = compare_nrmse(y_test, y_pred)
    return y_pred, nrmse

In [5]:
def experiment(args):
    loader = Loader(args)
    partition = loader.get_dataset()
    
    ts = time.time()
    model, history = encoder_decoder(partition, args)
    te = time.time()
    
    print('Took {:2.2f} sec for training the model'.format(te-ts))
    
    y_pred, test_acc = test(partition, model, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = history.history['loss']
    result['val_losses'] = history.history['val_loss']
    result['train_accs'] = history.history['accuracy']
    result['val_accs'] = history.history['val_accuracy']
    result['train_acc'] = np.mean(history.history['accuracy'])
    result['val_acc'] = np.mean(history.history['val_accuracy'])
    result['test_nrmse'] = test_acc
    result['y_test'] = np.array_str(partition['test'][1])
    result['y_pred'] = np.array_str(y_pred)
    
    setattr(args, 'features', str(args.features))
    return vars(args), model, result

### Manage Experiment

In [6]:
def get_filepath(setting, filetype):
    exp_name = setting['exp_name']
    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    
    file_format = ""
    directory = ""
    if filetype == FileType.MODEL:
        file_format = 'h5'
        directory = 'models'
    elif filetype == FileType.RESULT:
        file_format = 'json'
        directory = 'results'
        
    filename = "%s-%s.%s" % (exp_name, hash_key, file_format)
    filepath = os.path.join(Path(os.getcwd()).parent.parent, directory, filename)
    return filepath
        
def save_exp_model(setting, model):
    filepath = get_filepath(setting, FileType.MODEL)
    model.save(filepath)

def save_exp_result(setting, result):
    filepath = get_filepath(setting, FileType.RESULT)
    
    result.update(setting)
    with open(filepath, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = os.path.join(Path(os.getcwd()).parent.parent, "results")
    filenames = [f for f in listdeir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result)
    return df

In [7]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20190820"
args.end_date = "20200809"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = 'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.SUNSHINE,
                 FeatureType.HUMIDITY,
                 FeatureType.WIND_SPEED,
                 FeatureType.VISIBILITY,
                 FeatureType.GROUND_TEMPERATURE,
                 FeatureType.WIND_DIRECTION,
                 FeatureType.STEAM_PRESSURE,
                 FeatureType.TEMPERATURE,
                 FeatureType.PRECIPITATION,
                 FeatureType.DEW_POINT_TEMPERATURE,
                 FeatureType.ATMOSPHERIC_PRESSURE]

for i in range(len(list_var1)):
    sub_list = list_var1[:i + 1]

    setattr(args, name_var1, str(sub_list))
    print(args)

    setattr(args, name_var1, sub_list)   
    setting, model, result = experiment(deepcopy(args))
    save_exp_model(setting, model)
    save_exp_result(setting, result)

Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
Epoch 1/256
5/5 [==============================] - 1s 110ms/step - loss: 1316954240.0000 - accuracy: 0.1986 - val_loss: 855095296.0000 - val_accuracy: 0.0050
Epoch 2/256
5/5 [==============================] - 0s 9ms/step - loss: 1316473600.0000 - accuracy: 0.0077 - val_loss: 854939840.0000 - val_accuracy: 9.9206e-04
Epoch 3/256
5/5 [==============================] - 0s 8ms/step - loss: 1316354048.0000 - accuracy: 0.0042 - val_loss: 854866688.0000 - val_accuracy: 0.0000e+00
Epoch 4/256
5/5 [==============================] - 0s 9ms/step - loss: 1316287616.0000 - accuracy: 0.0030 - val_loss: 854813952.0000 - val_accuracy: 0.0000e+00
Epoch 5/256
5/5 [=========

Epoch 49/256
5/5 [==============================] - 0s 8ms/step - loss: 1314331264.0000 - accuracy: 0.0030 - val_loss: 852857152.0000 - val_accuracy: 0.0030
Epoch 50/256
5/5 [==============================] - 0s 9ms/step - loss: 1314291840.0000 - accuracy: 0.0086 - val_loss: 852812672.0000 - val_accuracy: 0.0040
Epoch 51/256
5/5 [==============================] - 0s 8ms/step - loss: 1314248064.0000 - accuracy: 0.0187 - val_loss: 852771392.0000 - val_accuracy: 0.0000e+00
Epoch 52/256
5/5 [==============================] - 0s 9ms/step - loss: 1314206720.0000 - accuracy: 0.0000e+00 - val_loss: 852728704.0000 - val_accuracy: 0.0000e+00
Epoch 53/256
5/5 [==============================] - 0s 9ms/step - loss: 1314163968.0000 - accuracy: 0.0042 - val_loss: 852681984.0000 - val_accuracy: 0.0020
Epoch 54/256
5/5 [==============================] - 0s 9ms/step - loss: 1314118400.0000 - accuracy: 0.0104 - val_loss: 852640832.0000 - val_accuracy: 0.0079
Epoch 55/256
5/5 [============================

5/5 [==============================] - 0s 9ms/step - loss: 1312109568.0000 - accuracy: 0.0384 - val_loss: 850618496.0000 - val_accuracy: 0.0208
Epoch 102/256
5/5 [==============================] - 0s 9ms/step - loss: 1312064768.0000 - accuracy: 0.0379 - val_loss: 850577280.0000 - val_accuracy: 0.0208
Epoch 103/256
5/5 [==============================] - 0s 9ms/step - loss: 1312023808.0000 - accuracy: 0.0368 - val_loss: 850535360.0000 - val_accuracy: 0.0238
Epoch 104/256
5/5 [==============================] - 0s 10ms/step - loss: 1311980928.0000 - accuracy: 0.0376 - val_loss: 850495872.0000 - val_accuracy: 0.0278
Epoch 105/256
5/5 [==============================] - 0s 9ms/step - loss: 1311938688.0000 - accuracy: 0.0384 - val_loss: 850450944.0000 - val_accuracy: 0.0258
Epoch 106/256
5/5 [==============================] - 0s 9ms/step - loss: 1311895552.0000 - accuracy: 0.0365 - val_loss: 850405696.0000 - val_accuracy: 0.0169
Epoch 107/256
5/5 [==============================] - 0s 9ms/step 

Epoch 153/256
5/5 [==============================] - 0s 8ms/step - loss: 1309870080.0000 - accuracy: 0.0530 - val_loss: 848372416.0000 - val_accuracy: 0.0308
Epoch 154/256
5/5 [==============================] - 0s 9ms/step - loss: 1309825664.0000 - accuracy: 0.0519 - val_loss: 848332672.0000 - val_accuracy: 0.0288
Epoch 155/256
5/5 [==============================] - 0s 9ms/step - loss: 1309785600.0000 - accuracy: 0.0544 - val_loss: 848296448.0000 - val_accuracy: 0.0347
Epoch 156/256
5/5 [==============================] - 0s 9ms/step - loss: 1309746048.0000 - accuracy: 0.0464 - val_loss: 848249728.0000 - val_accuracy: 0.0248
Epoch 157/256
5/5 [==============================] - 0s 9ms/step - loss: 1309704192.0000 - accuracy: 0.0525 - val_loss: 848204928.0000 - val_accuracy: 0.0268
Epoch 158/256
5/5 [==============================] - 0s 9ms/step - loss: 1309658624.0000 - accuracy: 0.0555 - val_loss: 848159872.0000 - val_accuracy: 0.0238
Epoch 159/256
5/5 [==============================] -

Epoch 205/256
5/5 [==============================] - 0s 9ms/step - loss: 1307660544.0000 - accuracy: 0.0646 - val_loss: 846152832.0000 - val_accuracy: 0.0347
Epoch 206/256
5/5 [==============================] - 0s 9ms/step - loss: 1307615232.0000 - accuracy: 0.0695 - val_loss: 846109184.0000 - val_accuracy: 0.0367
Epoch 207/256
5/5 [==============================] - 0s 9ms/step - loss: 1307572224.0000 - accuracy: 0.0712 - val_loss: 846073472.0000 - val_accuracy: 0.0536
Epoch 208/256
5/5 [==============================] - 0s 9ms/step - loss: 1307529728.0000 - accuracy: 0.0712 - val_loss: 846021184.0000 - val_accuracy: 0.0278
Epoch 209/256
5/5 [==============================] - 0s 9ms/step - loss: 1307487744.0000 - accuracy: 0.0706 - val_loss: 845985472.0000 - val_accuracy: 0.0526
Epoch 210/256
5/5 [==============================] - 0s 8ms/step - loss: 1307445248.0000 - accuracy: 0.0720 - val_loss: 845933888.0000 - val_accuracy: 0.0446
Epoch 211/256
5/5 [==============================] -

...

5/5 [==============================] - 0s 9ms/step - loss: 1315492352.0000 - accuracy: 0.0000e+00 - val_loss: 854020096.0000 - val_accuracy: 0.0000e+00
Epoch 22/256
5/5 [==============================] - 0s 9ms/step - loss: 1315448576.0000 - accuracy: 0.0000e+00 - val_loss: 853979264.0000 - val_accuracy: 0.0000e+00
Epoch 23/256
5/5 [==============================] - 0s 9ms/step - loss: 1315407616.0000 - accuracy: 0.0000e+00 - val_loss: 853932224.0000 - val_accuracy: 0.0000e+00
Epoch 24/256
5/5 [==============================] - 0s 9ms/step - loss: 1315361408.0000 - accuracy: 0.0000e+00 - val_loss: 853889408.0000 - val_accuracy: 0.0000e+00
Epoch 25/256
5/5 [==============================] - 0s 9ms/step - loss: 1315319040.0000 - accuracy: 0.0000e+00 - val_loss: 853848448.0000 - val_accuracy: 0.0000e+00
Epoch 26/256
5/5 [==============================] - 0s 9ms/step - loss: 1315277568.0000 - accuracy: 0.0000e+00 - val_loss: 853802816.0000 - val_accuracy: 0.0000e+00
Epoch 27/256
5/5 [=====

Epoch 71/256
5/5 [==============================] - 0s 9ms/step - loss: 1313336448.0000 - accuracy: 0.0000e+00 - val_loss: 851854976.0000 - val_accuracy: 0.0000e+00
Epoch 72/256
5/5 [==============================] - 0s 9ms/step - loss: 1313299968.0000 - accuracy: 0.0000e+00 - val_loss: 851840832.0000 - val_accuracy: 0.0000e+00
Epoch 73/256
5/5 [==============================] - 0s 9ms/step - loss: 1313258240.0000 - accuracy: 0.0000e+00 - val_loss: 851763776.0000 - val_accuracy: 0.0000e+00
Epoch 74/256
5/5 [==============================] - 0s 8ms/step - loss: 1313204864.0000 - accuracy: 0.0000e+00 - val_loss: 851717568.0000 - val_accuracy: 0.0000e+00
Epoch 75/256
5/5 [==============================] - 0s 10ms/step - loss: 1313159552.0000 - accuracy: 0.0000e+00 - val_loss: 851679040.0000 - val_accuracy: 0.0000e+00
Epoch 76/256
5/5 [==============================] - 0s 9ms/step - loss: 1313120384.0000 - accuracy: 0.0000e+00 - val_loss: 851634496.0000 - val_accuracy: 0.0000e+00
Epoch 77/

5/5 [==============================] - 0s 9ms/step - loss: 1311211008.0000 - accuracy: 0.0000e+00 - val_loss: 849716544.0000 - val_accuracy: 0.0000e+00
Epoch 121/256
5/5 [==============================] - 0s 9ms/step - loss: 1311169280.0000 - accuracy: 0.0000e+00 - val_loss: 849672960.0000 - val_accuracy: 0.0000e+00
Epoch 122/256
5/5 [==============================] - 0s 9ms/step - loss: 1311125632.0000 - accuracy: 0.0000e+00 - val_loss: 849629568.0000 - val_accuracy: 0.0000e+00
Epoch 123/256
5/5 [==============================] - 0s 10ms/step - loss: 1311082368.0000 - accuracy: 0.0000e+00 - val_loss: 849583552.0000 - val_accuracy: 0.0000e+00
Epoch 124/256
5/5 [==============================] - 0s 9ms/step - loss: 1311074944.0000 - accuracy: 0.0000e+00 - val_loss: 849600000.0000 - val_accuracy: 0.0000e+00
Epoch 125/256
5/5 [==============================] - 0s 9ms/step - loss: 1311008640.0000 - accuracy: 0.0000e+00 - val_loss: 849498688.0000 - val_accuracy: 0.0000e+00
Epoch 126/256
5/5

5/5 [==============================] - 0s 9ms/step - loss: 1309092352.0000 - accuracy: 0.0000e+00 - val_loss: 847585472.0000 - val_accuracy: 0.0000e+00
Epoch 170/256
5/5 [==============================] - 0s 10ms/step - loss: 1309047808.0000 - accuracy: 0.0000e+00 - val_loss: 847554176.0000 - val_accuracy: 0.0000e+00
Epoch 171/256
5/5 [==============================] - 0s 9ms/step - loss: 1309001216.0000 - accuracy: 0.0000e+00 - val_loss: 847497088.0000 - val_accuracy: 0.0000e+00
Epoch 172/256
5/5 [==============================] - 0s 9ms/step - loss: 1308956032.0000 - accuracy: 0.0000e+00 - val_loss: 847452288.0000 - val_accuracy: 0.0000e+00
Epoch 173/256
5/5 [==============================] - 0s 9ms/step - loss: 1308915328.0000 - accuracy: 0.0000e+00 - val_loss: 847407680.0000 - val_accuracy: 0.0000e+00
Epoch 174/256
5/5 [==============================] - 0s 9ms/step - loss: 1308872192.0000 - accuracy: 0.0000e+00 - val_loss: 847369280.0000 - val_accuracy: 0.0000e+00
Epoch 175/256
5/5

5/5 [==============================] - 0s 10ms/step - loss: 1306978048.0000 - accuracy: 0.0044 - val_loss: 845492928.0000 - val_accuracy: 0.0000e+00
Epoch 219/256
5/5 [==============================] - 0s 9ms/step - loss: 1306937728.0000 - accuracy: 0.0039 - val_loss: 845431104.0000 - val_accuracy: 0.0000e+00
Epoch 220/256
5/5 [==============================] - 0s 9ms/step - loss: 1306891776.0000 - accuracy: 0.0042 - val_loss: 845387968.0000 - val_accuracy: 0.0000e+00
Epoch 221/256
5/5 [==============================] - 0s 9ms/step - loss: 1306845696.0000 - accuracy: 0.0080 - val_loss: 845331520.0000 - val_accuracy: 0.0000e+00
Epoch 222/256
5/5 [==============================] - 0s 9ms/step - loss: 1306804096.0000 - accuracy: 0.0017 - val_loss: 845295488.0000 - val_accuracy: 0.0000e+00
Epoch 223/256
5/5 [==============================] - 0s 9ms/step - loss: 1306758400.0000 - accuracy: 0.0099 - val_loss: 845241216.0000 - val_accuracy: 0.0000e+00
Epoch 224/256
5/5 [======================